In [42]:
import googleapiclient.discovery
from IPython.display import JSON, display
import json
import pandas as pd

In [43]:
f1 = open('apikey.txt', 'r')
api_key = f1.read()

api_service_name = "youtube"
api_version = "v3"

In [44]:
!pip install --upgrade google-api-python-client

In [45]:
channel_ids = ['UCX6OQ3DkcsbYNE6H8uQQuVA'] # MR Beast channelid # Add more channel ids

In [46]:
youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
response = request.execute()

print(json.dumps(response, indent=2))
#display(JSON(response))

{
  "kind": "youtube#channelListResponse",
  "etag": "R5Lt5Bs938NUIvj_aPwjieifVcc",
  "pageInfo": {
    "totalResults": 1,
    "resultsPerPage": 5
  },
  "items": [
    {
      "kind": "youtube#channel",
      "etag": "Y5IZQtLWWOlO40SUrqyaVilyc7c",
      "id": "UCX6OQ3DkcsbYNE6H8uQQuVA",
      "snippet": {
        "title": "MrBeast",
        "description": "Go Watch Beast Games! https://unfur.ly/BeastGames\nSUBSCRIBE FOR A COOKIE!\n\nAccomplishments:\n- Raised $20,000,000 To Plant 20,000,000 Trees\n- Removed 30,000,000 pounds of trash from the ocean\n- Helped 2,000 people walk again\n- Helped 1,000 blind people see\n- Helped 1,000 deaf people hear\n- Built wells in Africa\n- Built and gave away 100 houses\n- Adopted every dog in a shelter (twice)\n- Given millions to charity\n- Started my own snack company Feastables\n- Started my own software company Viewstats\n- Started Lunchly, a tasty, better-for-you lunch option\n- Gave away a private island (twice)\n- Gave away 1 million meals\n-

In [47]:
channels_data = []
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    items = response['items']
    for item in items:
        channel_stats = {
                         'channel_name' : item['snippet']['title'],
                         'subscribers' : item['statistics']['subscriberCount'],
                         'views' : item['statistics']['viewCount'],
                         'total_videos' : item['statistics']['videoCount'],
                         'playlist_id' : item['contentDetails']['relatedPlaylists']['uploads']
                        }
        channels_data.append(channel_stats)
    return channels_data

In [48]:
all_channel_data =[]
for channel_id in channel_ids:
    all_channel_data.append(get_channel_stats(youtube, channel_id))

In [49]:
all_channel_data

[[{'channel_name': 'MrBeast',
   'subscribers': '346000000',
   'views': '70328115448',
   'total_videos': '842',
   'playlist_id': 'UUX6OQ3DkcsbYNE6H8uQQuVA'}]]

In [50]:
playlist_ids = []
for channel_data in all_channel_data:
    for k,v in channel_data[0].items():
      if k == 'playlist_id':
        playlist_ids.append(v)

def getVideoIds(youtube, playlist_id):
  videoIds = []
  for playlist_id in playlist_ids:
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults=50
      )
    response = request.execute()
    items = response.get('items')
    for item in items:
        video_id = item['contentDetails']['videoId']
        videoIds.append(video_id)

    nextPageToken = response.get('nextPageToken')
    while nextPageToken is not None:
      request = youtube.playlistItems().list(
        part = "snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50,
        pageToken = nextPageToken
      )
      response = request.execute()
      items = response.get('items')
      for item in items:
          video_id = item['contentDetails']['videoId']
          videoIds.append(video_id)
      nextPageToken = response.get('nextPageToken')
    return videoIds

videoIds = getVideoIds(youtube, playlist_ids)
print(len(videoIds))

842


In [51]:
videoIds[0]
request = youtube.videos().list(
        part="contentDetails, snippet, statistics, topicDetails",
        id=videoIds[:5]
    )
response = request.execute()

print(json.dumps(response, indent=2))

{
  "kind": "youtube#videoListResponse",
  "etag": "OHw6mz17L9mAjQxqnF1TD55RKH0",
  "items": [
    {
      "kind": "youtube#video",
      "etag": "RO2esoMxmtnd10UOuPLf0dima0s",
      "id": "TRxMLLuWIhw",
      "snippet": {
        "publishedAt": "2025-01-17T17:00:00Z",
        "channelId": "UCX6OQ3DkcsbYNE6H8uQQuVA",
        "title": "Subscribe For An iPhone",
        "description": "Terms and Conditions: https://bit.ly/MrBeastIphone",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/TRxMLLuWIhw/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/TRxMLLuWIhw/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/TRxMLLuWIhw/hqdefault.jpg",
            "width": 480,
            "height": 360
          },
          "standard": {
            "url": "https://i.ytimg.com/vi

In [52]:
def get_video_details(youtube, video_ids):
  videos = []
  i=0
  while i < len(videoIds):
    request = youtube.videos().list(
          part="contentDetails, snippet, statistics",
          id=videoIds[i:i+50]
      )
    response = request.execute()
    stats = {'snippet' :['publishedAt', 'title', 'channelTitle','tags','description'],
            'statistics' :['viewCount', 'likeCount', 'commentCount'],
            'contentDetails' :['duration','definition','caption']}

    items = response['items']
    for item in items:
      video = {}
      try:
        video['video_id'] = item['id']
      except:
        video['video_id'] = None

      for k in stats.keys():
        for v in stats[k]:
          try:
              video[v]=item[k][v]
          except:
              video[v] = None
      videos.append(video)
    i+=50
  return pd.DataFrame(videos)


In [53]:
videos_data = get_video_details(youtube, videoIds)
videos_data.head()

,video_id,publishedAt,title,channelTitle,tags,description,viewCount,likeCount,commentCount,duration,definition,caption
0,TRxMLLuWIhw,2025-01-17T17:00:00Z,Subscribe For An iPhone,MrBeast,None,Terms and Conditions: https://bit.ly/MrBeastIp...,23088137,2190111,89597,PT17S,hd,false
1,SM66GDRyIVY,2025-01-11T17:00:02Z,"I Helped 2,000 People Walk Again",MrBeast,None,Thank you for making this video possible\nROMP...,67357807,2363014,44584,PT15M31S,hd,true
2,EvqtfK_RaRg,2025-01-10T17:00:00Z,This obstacle course is WILD,MrBeast,None,,58137997,2015291,2942,PT32S,hd,false
3,3cWm9B_0_kI,2024-12-27T19:00:00Z,How Many People To Stop Ronaldo?,MrBeast,None,,150983496,6478397,15013,PT21S,hd,false
4,qtWqHIx8Z-o,2024-12-22T17:00:01Z,"Last Person Standing Wins $10,000",MrBeast,None,,197577136,7378225,5517,PT48S,hd,false


In [54]:
videos_data.shape

(842, 12)

In [55]:
comments = []
def get_channel_comments(youtube, videoIds):
  for video_id in videoIds:
    try:
      request = youtube.commentThreads().list(
              part="snippet, replies",
              videoId='TRxMLLuWIhw'
          )
      response = request.execute()
    except:
      continue
    items = response['items']
    # print(json.dumps(response, indent=2))
    for item in items:
      comment = item.get('snippet').get('topLevelComment').get('snippet').get('textOriginal')
      comment_entry = {'videoId' :video_id, 'comment':comment}
      comments.append(comment_entry)

  return pd.DataFrame(comments)

In [56]:
comments_df = get_channel_comments(youtube, videoIds)
comments_df.head()

,videoId,comment
0,TRxMLLuWIhw,Please make sure you have an email address on ...
1,TRxMLLuWIhw,"WHYYY , IM FROM RUSSIA😭😭 ( check the rules)"
2,TRxMLLuWIhw,Bismillah iya allah😭🤲
3,TRxMLLuWIhw,New sub!!!!
4,TRxMLLuWIhw,Ok brother 😊


In [57]:
videos_data.to_csv('videos_data.csv')
comments_df.to_csv('comments_data.csv')